# HRS Covid Data Exploration

In this notebook, I explore the Health and Retirement Study (HRS) COVID-19 Project Dataset, focusing on how different racial and socioeconomic groups delayed their access to medical care in the midst of the Covid-19 pandemic.

+ Data Source: https://hrsdata.isr.umich.edu/data-products/public-survey-data
+ Questionnaires: https://hrs.isr.umich.edu/documentation/questionnaires
+ Tracker Codebook: https://hrs.isr.umich.edu/sites/default/files/meta/tracker/codebook/trk2018tr_ri.htm
+ RAND Codebook: https://hrs.isr.umich.edu/sites/default/files/meta/2018/core/codebook/h18_00.html
+ Notebook Styling: https://dadoverflow.com/2019/06/20/ten-things-i-like-to-do-in-jupyter-markdown/

<div class="alert alert-block alert-success">
    <h1>Table of Contents</h1>
    <ul>
        <li><a href=#7.29>7/29 - Predict Delayed Dental and Routine Delayed among blacks and hispanic Only</a></li>
        <li><a href=#7.16>7/16 - Predict Delayed Surgery and Examine Delay Reasons</a></li>
        <li><a href=#6.28>6/28 - Add Disease Burden Variable</a></li>
        <li><a href=#6.14>6/14 - Add Self-Reported-Health Variable</a></li>
        <li><a href=#6.2>6/2 - Create Covid Exposure and Covid Death Variables</a></li>
        <li><a href=#5.26>5/26 - Delay Dental Care and Routine Checkup</a></li>
        <li><a href=#5.24>5/24 - Create delayReason Variable</a></li>
        <li><a href=#5.20>5/20 - First Logistic Regression modeling delayed care</a></li>
        <li><a href=#5.19>5/19 - Data Cleaning and creating new race variables</a></li>
        <li><a href=#npwhereselect>np.where() and np.select()</a></li>
        <li><a href=#appendix>Appendix</a></li>
    </ul>
</div>

## Imports

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
df = pd.read_csv('cleaned_df.csv')
df.head()

In [ ]:
#converting df to a csv file
#df.to_csv('cleaned_df.csv', index=False)

## Utilities

In [ ]:
def runLogModel(data, dependentVar, predictorsList):
    predictorsString = " + ".join(predictorsList)
    
    import statsmodels.api as sm
    from statsmodels.formula.api import logit
    logit = logit(f"{dependentVar} ~ {predictorsString}", data=data).fit()
    print(logit.summary())

In [ ]:
def makeNewVar(newName, codeName, yesNum, isValid):
    df[newName] = np.select([df[codeName]==yesNum,df[codeName]<=isValid], [1, 0], default=np.nan)
    print(df[newName].value_counts(dropna=False))

In [ ]:
core_predictors = ["age", "edu_years", "male", "nonHispanicBlack", "nonHispanicOther", "HispanicAll"]
exposure_predictors = ["covidExposure", "covidDeath"]
health_predictors = ["selfRateHealth", "diseaseBurden"]
marital_predictors = ["divorced", "widowed", "neverMarried", "usBorn"]
insurance_predictors = ["medicare", "medicaidCurrent", "militaryHealth", "privateInsurance", "publicInsurance"]
to_predict = ["delayedCare", "RoutineDelayed", "DentalDelayed", "SurgeryDelayed"]

---

## 8/12 <a name='8.12'></a>
+ Make Working ? Variable W601
+ W552 Covid test ? Make Variable 
+ RAND hrs 2018: Extract Household income, household wealth, household members
+ manipulate household income: log(($income + 1) / sqrt(household members))
+ household wealth: find negative numbers: add most negative to EVERYONE and add 1. Apply the above log formula
+ Run model with everyone

In [ ]:
#read in RAND hrs 2018
rand_df = pd.read_stata('./h18e1a_STATA/h18e1a.dta')

In [ ]:
rand_df.head()

In [ ]:
#Was your work affected because of the coronavirus pandemic?
#1. yes 5. no 6. was not working 8,9 dont'know #1 and 5  means you were working, 6 means no
df["RCOVW601"].value_counts(dropna=False)

In [ ]:
#"Have you been tested for Covid 19"
# 1: Yes 5: no 8,9: dont-know
df["RCOVW552"].value_counts(dropna=False)

In [ ]:
makeNewVar("testedForCovid19", "RCOVW552", 1, 5)

In [ ]:
df["isWorking"] = np.select([df["RCOVW601"]==6,df["RCOVW601"]<=5], [0, 1], default=np.nan)

In [ ]:
df["isWorking"].value_counts(dropna=False)

In [ ]:
extras = ["nonHispanicWhite", "covidConcern", "delayReason", "isWorking", "testedForCovid19", "hhid", "pn"]

In [ ]:
df_to_export = df[core_predictors+exposure_predictors+health_predictors+marital_predictors+insurance_predictors+to_predict+extras]

In [ ]:
#household income:  H14IFTOT

In [ ]:
#household wealth: H14ATOTB

In [ ]:
#number of people living in household: H14HHRES

In [ ]:
rand_df = pd.read_stata('./randhrs1992_2018v1_STATA/randhrs1992_2018v1.dta', columns=["h14itot","h14atotb","h14hhres","hhid","pn"])

In [ ]:
rand_df["h14itot"].value_counts(dropna=False)

In [ ]:
rand_df["hhIncome"] = rand_df["h14itot"]
rand_df["hhWealth"] = rand_df["h14atotb"]
rand_df["hhMembers"] = rand_df["h14hhres"]

In [ ]:
rand_df["h14itot"].describe()

In [ ]:
df_to_export["hhid"]

In [ ]:
rand_df["hhid"]

In [ ]:
rand_df["hhid"].convert_dtypes(convert_integer=True, convert_string=False)
rand_df["pn"] = pd.to_numeric(rand_df["pn"], errors='coerce')

In [ ]:
rand_df["hhid"] = pd.to_numeric(rand_df["hhid"], errors='coerce')
rand_df["pn"] = pd.to_numeric(rand_df["pn"], errors='coerce')

In [ ]:
rand_df["pn"]

In [ ]:
rand_to_merge = rand_df[["hhid","pn","hhIncome","hhWealth","hhMembers"]]

In [ ]:
df_to_export.merge(rand_to_merge, on=["hhid", "pn"])

In [ ]:
rand_df

In [ ]:
df_to_export

## 7/29 <a name='7.29'></a>
+ Run model to predict routinedelay and dentaldelay among blacks and hispanic Only
+ Also look at reasons among each race group 

In [ ]:
dfBlack = df[df['nonHispanicBlack']==1]

In [ ]:
dfBlack['nonHispanicBlack'].value_counts()

In [ ]:
runLogModel(dfBlack, "RoutineDelayed",["age", "edu_years", "male"] +exposure_predictors+marital_predictors+health_predictors)

In [ ]:
runLogModel(dfBlack, "DentalDelayed",["age", "edu_years", "male"] +exposure_predictors+marital_predictors+health_predictors)

In [ ]:
dfHispanic = df[df['HispanicAll']==1]

In [ ]:
runLogModel(dfHispanic, "RoutineDelayed",["age", "edu_years", "male"] +exposure_predictors+marital_predictors+health_predictors)

In [ ]:
runLogModel(dfHispanic, "DentalDelayed",["age", "edu_years", "male"] +exposure_predictors+marital_predictors+health_predictors)

In [ ]:
dfBlack[dfBlack['DentalDelayed']==1]['delayReason'].value_counts(dropna=False, normalize=True)

In [ ]:
dfBlack[dfBlack['RoutineDelayed']==1]['delayReason'].value_counts(dropna=False, normalize=True)

In [ ]:
dfHispanic[dfHispanic['DentalDelayed']==1]['delayReason'].value_counts(dropna=False, normalize=True)

In [ ]:
dfHispanic[dfHispanic['RoutineDelayed']==1]['delayReason'].value_counts(dropna=False, normalize=True)

## 7/16 <a name='7.16'></a>
+ SurgeryDelay run model
+ preventative care (routinedelay) reasons
+ dental delay reasons

In [ ]:
#change some NAN to 0 if delayCare var is 0
df["delayedSurgery"] = np.where(df['delayedCare']==0, 5, df['RCOVW582'])
df['SurgeryDelayed'] = np.select([df['delayedSurgery']==5,df['delayedSurgery']==1], [0,1], default=np.nan)
df['SurgeryDelayed'].value_counts(dropna=False)

In [ ]:
runLogModel(df, "SurgeryDelayed",core_predictors+exposure_predictors+marital_predictors+health_predictors)

### delayReason
1. COULDN'T AFFORD IT
2. COULDN'T GET AN APPOINTMENT
3. THE CLINIC/HOSPITAL/DOCTOR'S OFFICE CANCELLED, CLOSED, OR SUGGESTED RESCHEDULING
4. DECIDED IT COULD WAIT 
5. WAS AFRAID TO GO \
7 OTHER (SPECIFY) (W581) \
8 DON'T KNOW \
9 REFUSe

In [ ]:
df[df['RoutineDelayed']==1]['delayReason'].value_counts(dropna=False, normalize=True)

In [ ]:
df[df['DentalDelayed']==1]['delayReason'].value_counts(dropna=False, normalize=True)

## 6/28 <a name='6.28'></a>
+ Disease burden (0-4)
+ publicInsurance one variable`
+ log model defaults? weights, sampling
    + https://www.statsmodels.org/devel/examples/notebooks/generated/glm_weights.html
    + there is a freq_weights or var_weights parameter for the model

In [ ]:
df['diseaseBurden'] = df["HBP"] + df["Stroke"] + df["heartDisease"] + df["Diabetes"]
df['diseaseBurden'].value_counts(dropna=False)

#make public insurance variable (have it or not)
#at least one yes = 1, all no = 0, missing = np.nan
yes_conditions = (df['medicare']==1) | (df['medicaidCurrent']==1)|(df['militaryHealth']==1)
no_conditions = (df['medicare']==0) & (df['medicaidCurrent']==0)&(df['militaryHealth']==0)
conditions_list = [yes_conditions, no_conditions]
df["publicInsurance"] = np.select( conditions_list, [1,0],default=np.nan )
df["publicInsurance"].value_counts(dropna=False)

I guess if you have a missing value somewhere, I can't really assign you a disease burden. Fortnately, only max 10 people fall in this boat

In [ ]:
# df['HBP'].value_counts(dropna=False)
# df['Diabetes'].value_counts(dropna=False)
# df['Stroke'].value_counts(dropna=False)
# df['heartDisease'].value_counts(dropna=False)
# df[df['HBP'].isna()]
#checking publicInsurance variable
#df[["medicare","medicaidCurrent", "militaryHealth","publicInsurance"]].tail(20)

In [ ]:
runLogModel(df, "DentalDelayed", new_vars+core_predictors+exposure_predictors+marital_predictors+health_predictors)

## 6/14  <a name='6.14'></a>
+ Find self reported health, chronic illnesses (heart disease, hypertension, diabetes, stroke), health insurance (public or private)?, marital status (seperate into divorced, never married, widowed), born in US? Run with Dental Care compare with delayedCare

In [ ]:
#Various Health Condition Variables
df["selfRateHealth"] = np.where(df["RC001"]>5, np.nan,df["RC001"])
df['HBP'] = np.select([df['RC005']==1,df['RC005']==8], [1,np.nan], default=0)
df['Diabetes'] = np.select([df['RC010']==1,df['RC010']==8], [1,np.nan], default=0)
df['Stroke'] = np.select([df['RC053']==1,df['RC053']==8], [1,np.nan], default=0)
df['heartDisease'] = np.select([df['RC036']==1,df['RC036']==8], [1,np.nan], default=0)
df['heartDisease'].value_counts(dropna=False)

In [ ]:
#marital status and usborn
df["divorced"] = np.select([df['qmarst']==2,df['qmarst']<=4], [1, 0], default=np.nan)
df["widowed"] = np.select([df['qmarst']==3,df['qmarst']<=4], [1, 0], default=np.nan)
df["neverMarried"] = np.select([df['qmarst']==4,df['qmarst']<=4], [1, 0], default=np.nan)

df["usBorn"] = np.select([df['usborn']==1,df['usborn']==5], [1, 0], default=np.nan)

In [ ]:
def makeNewVar(newName, codeName, yesNum, isValid):
    df[newName] = np.select([df[codeName]==yesNum,df[codeName]<=isValid], [1, 0], default=np.nan)
    print(df[newName].value_counts(dropna=False))

makeNewVar("medicare", "RC001", 1, 5)    
makeNewVar("medicaidEver", "RN005", 1, 5)
makeNewVar("medicaidCurrent", "RN006", 1, 5) #if you asnwered No to medicaid Ever, you are fake missing here
df["medicaidCurrent"] = np.where(df["medicaidEver"]==0, 0, df["medicaidCurrent"]) #adjusting for said problem

makeNewVar("militaryHealth", "RN007", 1, 5)
df["privateInsurance"] = np.select([df['RN023']==0,df['RN023']<=7], [0, 1], default=np.nan)

In [ ]:
df["qmarst"].value_counts()

In [ ]:
df["privateInsurance"].value_counts(dropna=False)

In [ ]:
def runLogModel(data, dependentVar, predictorsList):
    predictorsString = " + ".join(predictorsList)
    
    import statsmodels.api as sm
    from statsmodels.formula.api import logit
    logit = logit(f"{dependentVar} ~ {predictorsString}", data=data).fit()
    print(logit.summary())

In [ ]:
core_predictors = ["age", "edu_years", "male", "nonHispanicBlack", "nonHispanicOther", "HispanicAll"]
exposure_predictors = ["covidExposure", "covidDeath"]
health_predictors = ["selfRateHealth", "HBP", "Diabetes", "Stroke", "heartDisease"]
marital_predictors = ["divorced", "widowed", "neverMarried", "usBorn"]
insurance_predictors = ["medicare", "medicaidCurrent", "militaryHealth", "privateInsurance"]

### Models

In [ ]:
predictors = core_predictors + marital_predictors + health_predictors + exposure_predictors
runLogModel(df, "RoutineDelayed", predictors)

## 6/2  <a name='6.2'></a>
Create 2 new variables
+ CovidExposure: Recode 551 1&2 -> 1 others ->0; 574, 577 (3 total): add this predictor
+ covidDeath: same idea 
+ predict delayCare, delayRoutine, delayDental with the addition of these two new variables
+ google scholars: delayedcare during covid papers exist?

In [ ]:
#--- creating covidExposure ---
df["RCOVW551"].value_counts(dropna=False)
#551 is "YOU"; 574 is "anyone else in your household"; 577 is anyone else you know
df["covidExposure"] = np.where((df["RCOVW551"]<=2) | (df["RCOVW574"]==1) | (df["RCOVW577"]==1), 1, 0)


#---creating covidDeath---: 578
df["covidDeath"] =  np.where(df["RCOVW578"]==1, 1 , 0)
df["covidDeath"].value_counts(dropna=False)
df["covidExposure"].value_counts(dropna=False)

### Notes
+ Python’s or and and statements don’t work very well against Panda’s Series', so instead we need to use the bitwise or (|) and and (&)
+ put the multiple conditions each in parens

### Logistic Regression, RoutineDelayed w/ covidExposure and covidDeath

In [ ]:
import statsmodels.api as sm
from statsmodels.formula.api import logit
logit = logit("RoutineDelayed ~ covidExposure + covidDeath + age + edu_years + covidConcern + male + nonHispanicBlack + nonHispanicOther + HispanicAll",
              data = df).fit()
print(logit.summary())

### Logistic Regression, DentalDelayed w/ covidExposure and covidDeath

In [ ]:
import statsmodels.api as sm
from statsmodels.formula.api import logit
logit = logit("DentalDelayed ~ covidExposure + covidDeath + age + edu_years + covidConcern + male + nonHispanicBlack + nonHispanicOther + HispanicAll",
              data = df).fit()
print(logit.summary())

### Logistic Regression, delayedCare w/ covidExposure and covidDeath

In [ ]:
import statsmodels.api as sm
from statsmodels.formula.api import logit
logit = logit("delayedCare ~ covidExposure + covidDeath + age + edu_years + covidConcern + male + nonHispanicBlack + nonHispanicOther + HispanicAll",
              data = df).fit()
print(logit.summary())

## 5/26 <a name='5.26'></a>
+ Type of delay: w585
Recode: 3 -> 1 (Routine checkup or screening)
Recode: 8 or 9 -> NaN
Recode: all others ->0
+ w587 dental
delay = 1
all others (including no delay) = 0
+ Predicting delay care in dental and delay care in routing checkup in logistic
#### Creating new binary vars: RoutineDelayed and Dental Delayed

In [ ]:
#--- The PROCESS ---

#Type of Delay (routing checkup ?)
df["RCOVW585M1"].value_counts(dropna=False)
df["RoutineDelayed"] = np.select([df["RCOVW585M1"]==3, df["RCOVW585M1"]==8], [1,np.nan], default=0)
#Probably need to change some 0 to NaN..if the person had missing data for delay care column
df["RoutineDelayed"].value_counts(dropna=False)

#Dental Delay
df["RCOVW587"].value_counts(dropna=False)
df["DentalDelayed"] = np.select([df["RCOVW587"]==1, df["RCOVW585M1"]==8], [1,np.nan], default=0)
df["DentalDelayed"].value_counts()

#change some 0 to NaN..if the person had missing data for delay care column
df["RoutineDelayed"] = np.where(np.isnan(df["delayedCare"]), np.nan, df["RoutineDelayed"])
df["RoutineDelayed"].value_counts(dropna=False) #Corrected: 15 more nans

#same thing with dental delay
df["DentalDelayed"] = np.where(np.isnan(df["delayedCare"]), np.nan, df["DentalDelayed"])
df["DentalDelayed"].value_counts(dropna=False) #Corrected: 15 more nans

In [ ]:
#Putting it All Together: accounting for missing delayCare
#This is because the way I coded it, if you were NaN in the routineCare,you would get 0.
df["RoutineDelayed"] = np.select([df["RCOVW585M1"]==3, df["RCOVW585M1"]==8], [1,np.nan], default=0)
df["RoutineDelayed"] = np.where(np.isnan(df["delayedCare"]), np.nan, df["RoutineDelayed"])

df["DentalDelayed"] = np.select([df["RCOVW587"]==1, df["RCOVW585M1"]==8], [1,np.nan], default=0)
df["DentalDelayed"] = np.where(np.isnan(df["delayedCare"]), np.nan, df["DentalDelayed"])

### Logistic Regression, dependent var: RoutineDelayed

In [ ]:
import statsmodels.api as sm
from statsmodels.formula.api import logit
logit = logit("RoutineDelayed ~ age + edu_years + covidConcern + male + nonHispanicBlack + nonHispanicOther + HispanicAll",
              data = df).fit()
print(logit.summary())

### Logistic Regression, dependent var: DentalDelayed

In [ ]:
import statsmodels.api as sm
from statsmodels.formula.api import logit
logit = logit("DentalDelayed ~ age + edu_years + male + nonHispanicBlack + nonHispanicOther + HispanicAll",
              data = df).fit()
print(logit.summary())

## 5/24 <a name='5.24'></a>
### Creating delayReason Variable
1. COULDN'T AFFORD IT
2. COULDN'T GET AN APPOINTMENT
3. THE CLINIC/HOSPITAL/DOCTOR'S OFFICE CANCELLED, CLOSED, OR SUGGESTED RESCHEDULING
4. DECIDED IT COULD WAIT 
5. WAS AFRAID TO GO

7 OTHER (SPECIFY) (W581)\
8. DON'T KNOW\
9. REFUSE

In [ ]:
df["RCOVW580M1"].value_counts(dropna=False) 
#136 answered other!! uhhh

In [ ]:
#those who did delay care, but gave no reason: np.nan
#those who didn't delay care: i'll code as 0, because you're not actually "missing"
df["delayReason"] = np.where(df["delayedCare"], df["RCOVW580M1"], 0)

#response 8 and 9 are also considered "missing"
df["delayReason"] = np.where(df["delayReason"]>=8, np.nan, df["delayReason"])
df["delayReason"].value_counts(dropna=False)

In [ ]:
raceList=['nonHispanicWhite','nonHispanicBlack','nonHispanicOther', 'HispanicAll']
race_counts=[]
for race in raceList:
    race_counts.append(len(df[df[race]==1]))
print(race_counts)

In [ ]:
# Proportion of delayReason
for race in raceList:
    print(f"{race}: delayReason")
    slice = df[df[race]==1]
    print(slice["delayReason"].value_counts(normalize=True))

## 5/20 <a name='5.20'></a>
+ CovidConcern by Race Summary Stats
+ Mean age, gender distribution, schlyrs by race
+ Logistic Regression predicting delayCare binary variable

In [ ]:
raceList=['nonHispanicWhite','nonHispanicBlack','nonHispanicOther', 'HispanicAll']
for race in raceList:
    concern_mean = df[df[race]==1]["covidConcern"].mean()
    age_mean = df[df[race]==1]["age"].mean()
    edu_years_mean = df[df[race]==1]["edu_years"].mean()
    male_percentage = df[df[race]==1]["male"].mean()
    
    print(f"{race}:")
    print(f"cocern_mean: {concern_mean}")
    print(f"age_mean: {age_mean}")
    print(f"edu_years_mean: {edu_years_mean}")
    print(f"male_percentage: {male_percentage}")
    print('\n')

Doing the same thing in a much easier way by using the groupby() method with an aggregation function like .mean() to extract these features with ease

In [ ]:
df.groupby('nonHispanicWhite').mean()[['age','edu_years','covidConcern','male']]

In [ ]:
for race in raceList:
    print(race, "Means")
    print(df.groupby(race).mean()[['age','edu_years','covidConcern','male']].loc[1])
    print('\n')

### Logistic Regression: Depedent Var: delayedCare

Dependent Var: Delayed (1 or 0)
Independent Vars: Age, edu_years, gender, covid concern, race (no white)
How to Interpret results
Reason for delay var by race
 

In [ ]:
import statsmodels.api as sm
from statsmodels.formula.api import logit
logit = logit("delayedCare ~ age + edu_years + covidConcern + male + nonHispanicBlack + nonHispanicOther + HispanicAll",
              data = df).fit()
print(logit.summary())

### Logistic Model Interpretation
+ log(p/1-p) = b0 + b1*female + b2*read + b3*science
+ where p is probability of person delaying care

## 5/19 <a name='5.19'></a>
+ Create 4 new vars (binary): non-hispanic whites, non-hispanic blacks, hispanics, other
+ Convert gender var from 1s and 2s to Female = 1 Male = 0 -> then get summary stats (mean)
+ Concern about covid var (rcovv550) : code 11-100 as missing(NaN)
+ Recode delayed care rcovv579 var: 1= true, 0=false
+ Get percentage of each race that delayed\

--- For Later ---
+ relationship between concern and delay care
+ relationship between delay care and race
+ Pandas Summary Stats Docs: https://pandas.pydata.org/pandas-docs/stable/getting_started/intro_tutorials/06_calculate_statistics.html


### np.where() and np.select()
<a name='npwhereselect'></a>
https://www.dataquest.io/blog/tutorial-add-column-pandas-dataframe-based-on-if-else-condition/

+ Using np.select(): two arguments: a list of our conditions, and a correspding list of the value we’d like to assign to each row in our new column
+ Using np.where(): three arguments in sequence: the condition we’re testing for, the value to assign to our new column if that condition is true, and the value to assign if it is false
+ have to use & instead of and... why?

In [ ]:
# ----- For Reference -----
# Note: value_counts exclude nan values by default
df['race'].value_counts(dropna=False)
# df['hispanic'].value_counts(dropna=False)
# df['gender'].value_counts(dropna=False)

# ---covid concern---
# df["RCOVW550"].value_counts(dropna=False)

# ---delayed care---
# df["RCOVW579"].value_counts(dropna=False)

In [ ]:
df['HispanicAll'].value_counts()

## Creating new variables
+ nonHispanicWhite
+ nonHispanicBlack
+ nonHispanicOther
+ HispanicAll
+ male
+ covidConcern (cleaned rcovw550)
+ delayedCare (cleaned rcovw579)
+ age (cleaned qage)
+ edu_years (cleaned schlyrs)

In [ ]:
#Codings for hispanic: 1,2,3 YES; 5 NO; 0 UNKNOWN
df['nonHispanicWhite'] = np.where((df['race']==1) & (df['hispanic']==5), 1, 0)
df['nonHispanicBlack'] = np.where((df['race']==2) & (df['hispanic']==5), 1, 0)
df['nonHispanicOther'] = np.where((df['race']==7) & (df['hispanic']==5), 1, 0)
df['HispanicAll'] = np.where((df['hispanic']<=3) & (df['hispanic']>0) & (df['race']!=0), 1, 0)

#codings for gender: 1=male, 2=female 0=blank or unknown
df['male'] = np.select([df['gender']==1,df['gender']==2], [1,0], default=np.nan)

#coding for covid concern 550: question asked 1-10 where 10 is most concerned
df["covidConcern"] = np.where(df["RCOVW550"]<=10, df["RCOVW550"], np.nan)

#coding for 579 delay_care: 1=yes 5=no 8 dont know 9 refuse (no 9s in dataset)
df["delayedCare"] = np.select([df["RCOVW579"]==1, df["RCOVW579"]==5, df["RCOVW579"]==8],[1,0,np.nan])

#age in 2018: 999 means "no core interview this wave"
df['age'] = np.where(df['qage']==999, np.nan, df['qage'])

#years of formal schooling: max is 17
df['edu_years'] = np.where(df['schlyrs']==99, np.nan, df['schlyrs'])

In [ ]:
# #Basic Checks for new columns
# #basic check
# df[['nonHispanicWhite','nonHispanicBlack','nonHispanicOther','HispanicAll','race', 'hispanic']]
# df['male'].value_counts()
# df[["covidConcern", "RCOVW550"]]
# #check NaN
# df[df["RCOVW550"]>10].head()

In [ ]:
# ---Looking at Summary Stats
df['age'].describe()
df['edu_years'].describe()
#40% of dataset are males
df['male'].describe()
df["delayedCare"].value_counts(dropna=False)

### Percentage of each race that delayed care

In [ ]:
raceList=['nonHispanicWhite','nonHispanicBlack','nonHispanicOther', 'HispanicAll']
total = 0;
length = 0;
for var in raceList:
    length = len(df[df[var]==1])
    total += length
    print(f"{var} count: {length}")
print("Total Count:", total)

#3239 checks out because people who were missing in race data could also be missing in hispanic data
print("Some Math:",3266-25-18)

In [ ]:
percentage = 0;
delayed_length = 0;
delayedCareList=[];

for var in raceList:
    length = len(df[df[var]==1])
    delayed_length = len(df[(df[var]==1) & (df['delayedCare']==1)])
    percentage = delayed_length/length
    delayedCareList.append(percentage)
    print(f"Percentage of {var} that delayed care: ", percentage)

In [ ]:
sns.barplot(x=raceList,y=delayedCareList)
plt.tight_layout()
plt.title('Percentage that delayed care')

## EDA Visualizations <a name="visualizations"></a>

In [ ]:
sns.displot(df, x="qage")

In [ ]:
sns.catplot(x="race", y="covidConcern", data=df, kind="box")

In [ ]:
sns.catplot(x="race", y="covidConcern", data=df, kind="violin")

----------

<div class="alert alert-block alert-success">
    <h1>Appendix</h1>
    <a name="appendix"></a>
</div>

## Importing Variables of Interest from the Tracker File
This includes demographic data like age, education, race, and marital status

In [ ]:
# import tracker file and extracting variables of interest
trk = pd.read_stata('./trk2018tr_r.dta')
vars_of_interest = ['hhid', 'pn', 'birthyr', 'race', 'hispanic', 'gender', 'degree',
                       'secu', 'stratum', 'qalive', 'qage', 'schlyrs', 'qmarst', 'usborn']

tracker_df = trk[cars_of_interest]

#merge our current dataframe with tracker dataframe
merged_df = df.merge(tracker_df, on=['hhid','pn'])

#convert this df to a csv file
merged_df.to_csv('cleaned_df.csv', index=False)

## Importing Covid Data from .dta files (first 20 only)

In [ ]:
pwd

In [ ]:
#Read in all the Data and put first 10 files into one dataframe
import os

allData = []
file_path = 'C:\\Users\\olive\\Documents\\data-projects\\covid-HRS\\coviddta'

#os.walk() yields 3-tuples
for folder, sub_folders, files in os.walk(file_path):
   
    #print("Files Processed: ")
    for f in files:
        #print(f"\tfile: {f}")
        if(f[-5:] == "r.dta"):    #if filename extension is r.dta, read in the file
            df = pd.read_stata(f"./coviddta/{f}")
            allData.append(df)
            
from functools import reduce

df = reduce(lambda dfL, dfR : pd.merge(dfL, dfR, on=['hhid', 'pn']), allData[:20])
print("Number of files read:", len(allData))

## Merging ALL Covid data 
**Warning** Takes over 30 mins

In [ ]:
from functools import reduce
df = reduce(lambda dfL, dfR : pd.merge(dfL, dfR, on=['hhid', 'pn']), allData[:10])
df2 = reduce(lambda dfL, dfR : pd.merge(dfL, dfR, on=['hhid', 'pn']), allData[10:20])
df3 = reduce(lambda dfL, dfR : pd.merge(dfL, dfR, on=['hhid', 'pn']), allData[20:30])
df4 = df.merge(df2, on=['hhid', 'pn'])
df_final = df4.merge(df3,on=['hhid', 'pn'])
df_final.to_csv('allData.csv')

## Export dataframe as stata .dta file

In [ ]:
rand_df.to_stata('rand8-24.dta', version=117, write_index=False)

## Miscellaneous

In [ ]:
#changing hhid and pn from type object to int
df["hhid"].convert_dtypes(convert_integer=True, convert_string=False)
df["pn"] = pd.to_numeric(df["pn"], errors='coerce')

#convert object to int
#tracker_df["hhid"] = pd.to_numeric(tracker_df["hhid"], errors='coerce')
#tracker_df["pn"] = pd.to_numeric(tracker_df["pn"], errors='coerce')

In [ ]:
df["hhid"]

In [ ]:
df["pn"]

In [ ]:
rand_df["pn"] = pd.to_numeric(rand_df["pn"], errors='coerce')

In [ ]:
rand_df["pn"].astype('int64')